In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


In [26]:
file_location = 'https://github.com/fawrama/diabetes_detecion_model/blob/bca7a90295c9abb20ea23bac89129b2abc0a34a2/diabetes.csv?raw=true'
all_ds = pd.read_csv(file_location)
all_ds

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [27]:
x = all_ds.iloc[:,:14]
y = all_ds.iloc[:,-1:]

In [28]:
le =  preprocessing.LabelEncoder()
x = x.apply(le.fit_transform)

In [29]:
print(x.shape)
print('X-->: ',x)
print('Y-->: ',y)

(768, 9)
X-->:       Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin  BMI  \
0              6       86             22             28        0  123   
1              1       23             19             22        0   62   
2              8      121             17              0        0   30   
3              1       27             19             16       62   77   
4              0       75              4             28      102  209   
..           ...      ...            ...            ...      ...  ...   
763           10       39             25             41      108  118   
764            2       60             21             20        0  155   
765            5       59             22             16       71   58   
766            1       64             14              0        0   95   
767            1       31             21             24        0   98   

     DiabetesPedigreeFunction  Age  Outcome  
0                         350   29        1  
1              

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [31]:
x_train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
597,1,27,1,12,7,74,317,0,0
240,1,29,17,17,0,87,73,0,0
486,1,77,16,34,174,192,305,0,0
612,7,106,32,35,156,169,417,19,1
324,2,50,24,25,0,144,35,0,0
...,...,...,...,...,...,...,...,...,...
392,1,69,17,7,169,34,219,0,0
710,3,96,17,6,166,105,156,3,0
239,0,42,25,0,0,2,328,6,0
209,7,122,29,26,0,142,198,20,1


In [32]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
scaler.fit(x_test)

StandardScaler()

In [33]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10,10,10),max_iter=1000)
y_train=y_train.astype('int')
mlp.fit(x_train,y_train.values.ravel())

MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)

In [34]:
pred = mlp.predict(x_test)

In [35]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(pred,y_test))
print(classification_report(y_test,pred))


[[148   4]
 [  4  75]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       152
           1       0.95      0.95      0.95        79

    accuracy                           0.97       231
   macro avg       0.96      0.96      0.96       231
weighted avg       0.97      0.97      0.97       231



In [36]:
pip install keras-tuner --upgrade

In [37]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [38]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=64),
                               activation='relu'))
  #one output layer as our prediction would either be 1 or 0
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])),
        loss='mean_absolute_error',
        metrics=['accuracy'])
  return model

In [39]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=6,
    executions_per_trial=3,
    project_name='Diabetes prediction')

In [40]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 64, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 64, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001, 1e-05], 'ordered': True}


In [41]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.30)

In [43]:
tuner.search(x_train, y_train, epochs=4, validation_data=(x_test, y_test))

Trial 6 Complete [00h 00m 10s]
val_accuracy: 0.5656565626462301

Best val_accuracy So Far: 0.6969696879386902
Total elapsed time: 00h 00m 45s
INFO:tensorflow:Oracle triggered exit


In [44]:
tuner.results_summary()

Results summary
Results in ./Diabetes prediction
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 19
units_0: 160
units_1: 288
learning_rate: 0.001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: 32
units_16: 32
units_17: 32
units_18: 32
Score: 0.6969696879386902
Trial summary
Hyperparameters:
num_layers: 4
units_0: 352
units_1: 480
learning_rate: 0.0001
units_2: 352
units_3: 32
units_4: 288
units_5: 32
units_6: 480
units_7: 416
units_8: 352
units_9: 288
units_10: 288
units_11: 288
units_12: 416
units_13: 352
units_14: 160
units_15: 160
units_16: 416
units_17: 224
units_18: 352
Score: 0.6969696879386902
Trial summary
Hyperparameters:
num_layers: 4
units_0: 224
units_1: 288
learning_rate: 1e-05
units_2: 224
units_3: 416
units_4: 416
units_5: 32
units_6: 416
units_7: 96
units_8: 96
units_9: 224
units_10: 480
units_11: 352
units_12: 288
units_13: 416
